#### Тема: применение методов машинного обучения для предсказания биотоксичности

In [1]:
#загружаем все необходимые библиотеки
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
#записываем smiles уже известных ингибиторов BCR-ABL тирозинкиназы и преобразовывем их в fingerprints
nilotinib_smiles = "CC1=C(C=C(C=C1)C(=O)NC2=CC(=CC(=C2)C(F)(F)F)N3C=C(N=C3)C)NC4=NC=CC(=N4)C5=CN=CC=C5"
nilotinib_mol = Chem.MolFromSmiles(nilotinib_smiles)
nilotinib_fingerprint = AllChem.GetMorganFingerprintAsBitVect(nilotinib_mol, radius=2, nBits=1024)

imatinib_smiles = "CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5"
imatinib_mol = Chem.MolFromSmiles(imatinib_smiles)
imatinib_fingerprint = AllChem.GetMorganFingerprintAsBitVect(nilotinib_mol, radius=2, nBits=1024)

dasatinib_smiles = "CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(=N3)C)N4CCN(CC4)CCO"
dasatinib_mol = Chem.MolFromSmiles(dasatinib_smiles)
dasatinib_fingerprint = AllChem.GetMorganFingerprintAsBitVect(nilotinib_mol, radius=2, nBits=1024)

bosutinib_smiles = "CN1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CC(=C3NC4=CC(=C(C=C4Cl)Cl)OC)C#N)OC"
bosutinib_mol = Chem.MolFromSmiles(bosutinib_smiles)
bosutinib_fingerprint = AllChem.GetMorganFingerprintAsBitVect(nilotinib_mol, radius=2, nBits=1024)

ponatinib_smiles = "CC1=C(C=C(C=C1)C(=O)NC2=CC(=C(C=C2)CN3CCN(CC3)C)C(F)(F)F)C#CC4=CN=C5N4N=CC=C5"
ponatinib_mol = Chem.MolFromSmiles(ponatinib_smiles)
ponatinib_fingerprint = AllChem.GetMorganFingerprintAsBitVect(nilotinib_mol, radius=2, nBits=1024)

asciminib_smiles = "C1CN(CC1O)C2=C(C=C(C=N2)C(=O)NC3=CC=C(C=C3)OC(F)(F)Cl)C4=CC=NN4"
asciminib_mol = Chem.MolFromSmiles(asciminib_smiles)
asciminib_fingerprint = AllChem.GetMorganFingerprintAsBitVect(nilotinib_mol, radius=2, nBits=1024)

In [19]:
#создаём набор из ингибиторов для проверки качесства работы модели
inhibitor_test = []
fp_inhibitors = [nilotinib_fingerprint,
                imatinib_fingerprint,
                dasatinib_fingerprint,
                bosutinib_fingerprint,
                ponatinib_fingerprint,
                asciminib_fingerprint]
for fp in fp_inhibitors:
    fp_str = fp.ToBitString()
    print(fp_str + '\n')
    tmpX = np.array(list(fp_str),dtype=float)
    inhibitor_test.append(tmpX)
inhibitor_test = np.array(inhibitor_test)

0000000000000000000100000000000001000000000000000000000000000100100000000000000000001000001000000000000000000000001000000000000010000010100000000000000000000000000000000000000000000000000000010100000010000000000000000001000000000000000010000000000000000000010000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000010000100000100000000001000000000000001000001000000000000000000000000000011000000010000010000000000000010000010000000000000000000001000010000000000000000001100000000000000100000000000010000000000000001001000100000001100000001000000000000000100000000000000000000000000000000000000000000000000000010000000100000000100000100000000000000000000000010000000010000000000000000001000000000100000000000000000000000000000000001000000000000000100001001000001000001010000000000000000000001000000000010000001000000000000000000000000010000000000000000010000000000100000000000000000000000000010010000000000000000001000000000000000000000000010000000001000000000

Тут будет предсказываться гепатоксичность(токсичность для печени). В общем случае вещества по своей токсичности могут быть разделены на 6 классов:

    - Класс 1 (A): смертельно при проглатывании (LD50 ≤ 5).
    - Класс 2 (B): смертельно при проглатывании (5 < LD50 ≤ 50)
    - Класс 3 (C): токсичен при проглатывании (50 < LD50 ≤ 300).
    - Класс 4 (D): вреден при проглатывании (300 < LD50 ≤ 2000).
    - Класс 5 (E): может быть вреден при проглатывании (2000 < LD50 ≤ 5000)
    - Класс 6 (F): нетоксичный (LD50 > 5000).

In [3]:
#уже известные классы биотоксичности представленных выше ингибиторов
nilotinib_real_tox = 4
imatinib_real_tox = 2
dasatinib_real_tox = 4
bosutinib_real_tox = 4
ponatinib_real_tox = 5
asciminib_real_tox = 5

In [4]:
#считываем данные и выделяем смайлз
data = pd.read_csv('smiles_data.csv')
smiles = data['smiles']

#делим на тестовую и обучающую выборку
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
train_smiles = train_data['smiles']
test_smiles = test_data['smiles']

#создаём файл со всеми смайлз соединений обучающей выборки, чтобы определить их токсичность с помощью сервиса
with open('train_smiles.csv', 'w') as file:
    for item in train_smiles:
        file.write(item + '\n')


In [7]:
#with open('train_smiles.csv', 'w') as file:
#    for item in train_smiles:
#        file.write(item + '\n')

In [ ]:
#преобразовывем все смайлз в fingerprints
fingerprints_train = []
for smile in train_smiles:
    fingerprint = Chem.MolFromSmiles(smile)
    fingerprints_train.append(fingerprint)

In [ ]:
fingerprints_test = []
for smile in test_smiles:
    fingerprint = Chem.MolFromSmiles(smile)
    fingerprints_test.append(fingerprint)

In [7]:
#добавляем в тестовую и обучающую данные колонку с отпечатками
train_data['fingerprint'] = fingerprints_train
test_data['fingerprint'] = fingerprints_test

#убираем значения None из списков
fingerprints_train = [x for x in fingerprints_train if x is not None]
fingerprints_test = [x for x in fingerprints_test if x is not None]

In [8]:
#формируем train и test сеты так, чтобы можно было использовать модель
X = []
X_train = []
for fp in fingerprints_train:
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(fp, radius=2, nBits=1024)
    fp_str = fingerprint.ToBitString()
    tmpX = np.array(list(fp_str),dtype=float)
    X_train.append(tmpX)
X = X_train
X_train = np.array(X_train)

In [9]:
X_test = []
for fp in fingerprints_test:
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(fp, radius=2, nBits=1024)
    fp_str = fingerprint.ToBitString()
    tmpX = np.array(list(fp_str),dtype=float)
    X_test.append(tmpX)
    X.append(tmpX)
X_test = np.array(X_test)

In [10]:
X = np.array(X)

In [11]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

    В качестве тренировочной выборки имеем набор из 78823 канонических смайлз соединений (бинарными строками после обработки данных); 

    Для данных в используемом датасете не известны классы токсичночти, поэтому для дальнейшей работы с данными и проверки используемых моделей на тестовых данных можно использовать методы обучения без учителя или обучения с частичным контролем. В данном случае используем алгоритм кластеризации для группировки молекул на основе их структурных характеристик.

In [42]:
#from sklearn.cluster import KMeans
#from rdkit import Chem
#from rdkit.Chem import AllChem

# Кластеризация
#kmeans = KMeans(n_clusters=6)  # предполагаем 6 классов
#kmeans.fit(X_train)
#toxes_train = kmeans.labels_

In [12]:
#toxes_train

In [13]:
import random
toxes_train = []
targets = [1, 2, 3, 4, 5, 6]
weights = [0.1, 0.2, 0.1, 0.5, 0.4, 0.2]
amount = 78823
for i in range(0, 78823):
    tox = random.choices(targets, weights=weights, k=1)[0]
    toxes_train.append(tox)

In [14]:
with open('tox_classes.csv', 'w') as file:
    for item in toxes_train:
        file.write(str(item) + '\n')

In [15]:
toxes = pd.read_csv('tox_classes.csv')

In [17]:
len(toxes_train)

78823

Machine learning models that are commonly used to predict biotoxicity of low-molecular compounds:

    - Random forest classifier 
    В данной работе будет использована как уже готовая модель, так и своя её реализация
    - K-nearest neighbors(kNN)
    - Gradient Boosting: XGBoost, LightGBM, CatBoost
    модели, основанные на градиентном бустинге, могут обеспечивать отличные результаты и обычно превосходят случайные леса на сложных задачах. Они помогают повысить производительность модели за счет последовательного обучения деревьев на остатковых ошибках.
    
    - Extra Trees classifier

In [20]:
#соединяем тестовую выборку и валидационные соединения для удобства
X_both = np.concatenate((X_test, inhibitor_test))

In [21]:
#применяем классификатор Random forest
classifier_1 = RandomForestClassifier(random_state=42)
classifier_1.fit(X_train, toxes_train)
tox_score_RF = classifier_1.predict(X_both)

In [23]:
#!!!!!!!!!!применям написанный самостоятельно Random Forest (реализация будет написана ниже)
#my_rf_classifier = MyRandomForestClassifier()
#my_rf_classifier.fit(X_train, toxes_train)
#tox_score_MyRF = my_rf_classifier.predict(X_both)

In [24]:
#применяем классификатор KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, toxes_train)
tox_score_KNN = knn.predict(X_both)

In [39]:
#применяем XGBoost
#import xgboost as xgb
#xgb = xgb.XGBClassifier(use_label_encoded=False, eval_metric='logloss')
#xgb.fit(X_train, toxes_train)
#tox_score_XGB = xgb.predict(X_both)

In [25]:
#применяем Extra Trees Classifier
extra_trees = ExtraTreesClassifier(n_estimators=100, random_state=42)
extra_trees.fit(X_train, toxes_train)
tox_score_ET = extra_trees.predict(X_both)

In [26]:
#предсказываем класс для ингибиторов и вычисляем точность модели
test_labels = [nilotinib_real_tox,
               imatinib_real_tox,
               dasatinib_real_tox,
               bosutinib_real_tox,
               ponatinib_real_tox ,
               asciminib_real_tox]
inhibitors_tox_pred_RF = tox_score_RF[-6:]
inhibitors_tox_pred_KNN = tox_score_KNN[-6:]
inhibitors_tox_pred_ET = tox_score_ET[-6:]
#inhibitors_tox_pred_XGB = tox_score_XGB[-6:]
#inhibitors_tox_pred_MyRF = tox_score_MyRF[-6:]

Далее проведём оценку работы модели, сравнивая реальные значения классов токсичности для исследуемых ингибиторов:

    - nilotinib
    - imatinib
    - dasatinib
    - bosutinib
    - ponatininb
    - asciminib
 
с полученными моделями значениями.

Используем функцию `predict_proba` для получения вероятностных оценок принадлежности переданных объектов к каждому из классов. Это полезно, когда важно не только узнать, к какому классу относится объект, но и насколько уверенно модель делает это предсказание.

In [27]:
probs_rf = classifier_1.predict_proba(X_both)
probs_knn = knn.predict_proba(X_both)
probs_et = extra_trees.predict_proba(X_both)
#probs_xgb = xgb.predict_proba(X_both)
#probs_myrf = 

In [40]:
#probs_et[-6:]

In [31]:
import math
def count_entropy(y_real, y_proba):
    if len(y_real) != len(y_proba):
        raise ValueError("Длины y_real и y_proba должны совпадать.")
    
    entropy = 0
    for i in range(len(y_real)):
        if y_proba[i] > 0:
            entropy += -y_real[i] * math.log(y_proba[i])
    return entropy

Сюда надо пояснения добавить к тому, что тут вообще происходит

In [33]:
y_real = [4, 2, 4, 4, 5, 5]
y_real_encoded = [[0, 0, 0, 1, 0, 0],
                  [0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 1, 0]]
y_probas_rf = probs_rf[-6:]
y_probas_knn = probs_knn[-6:]
y_probas_et = probs_et[-6:]
#y_probas_myrf =
#y_probas

In [34]:
#Вывод полученный значений энтропии
print('Nilotinib: ')
print('Random forest: ', count_entropy(y_real_encoded[0], y_probas_rf[0]))
print('KNN: ', count_entropy(y_real_encoded[0], y_probas_knn[0]))
print('Extra Trees classifier: ', count_entropy(y_real_encoded[0], y_probas_et[0]))
#
print()

print('Imatinib: ')
print('Random Forest', count_entropy(y_real_encoded[1], y_probas_rf[1]))
print('KNN: ', count_entropy(y_real_encoded[1], y_probas_knn[1]))
print('Extra Trees classifier: ', count_entropy(y_real_encoded[1], y_probas_et[1]))
#
print()

print('Dasatinib: ')
print('Random forest: ', count_entropy(y_real_encoded[2], y_probas_rf[2]))
print('KNN: ', count_entropy(y_real_encoded[2], y_probas_knn[2]))
print('Extra Trees classifier: ', count_entropy(y_real_encoded[2], y_probas_et[2]))
#
print()

print('Bosutinib: ')
print('Random forest: ', count_entropy(y_real_encoded[3], y_probas_rf[3]))
print('KNN: ', count_entropy(y_real_encoded[3], y_probas_knn[3]))
print('Extra Trees classifier: ', count_entropy(y_real_encoded[3], y_probas_et[3]))
#
print()

print('Ponatininb: ')
print('Random forest: ', count_entropy(y_real_encoded[4], y_probas_rf[4]))
print('KNN: ', count_entropy(y_real_encoded[4], y_probas_knn[4]))
print('Extra Trees classifier: ', count_entropy(y_real_encoded[4], y_probas_et[4]))
#
print()

print('Asciminib: ')
print('Random forest: ', count_entropy(y_real_encoded[5], y_probas_rf[5]))
print('KNN: ', count_entropy(y_real_encoded[5], y_probas_knn[5]))
print('Extra Trees classifier: ', count_entropy(y_real_encoded[4], y_probas_et[4]))
#

Nilotinib: 
Random forest:  1.1086626245216111
KNN:  1.6094379124341003
Extra Trees classifier:  1.2378743560016174

Imatinib: 
Random Forest 2.120263536200091
KNN:  1.6094379124341003
Extra Trees classifier:  1.6094379124341003

Dasatinib: 
Random forest:  1.1086626245216111
KNN:  1.6094379124341003
Extra Trees classifier:  1.2378743560016174

Bosutinib: 
Random forest:  1.1086626245216111
KNN:  1.6094379124341003
Extra Trees classifier:  1.2378743560016174

Ponatininb: 
Random forest:  1.7147984280919266
KNN:  1.6094379124341003
Extra Trees classifier:  1.9661128563728327

Asciminib: 
Random forest:  1.7147984280919266
KNN:  1.6094379124341003
Extra Trees classifier:  1.9661128563728327


##### Оценка точности модели с помощью метрик


In [36]:
# Random forest model
accuracy_rf = accuracy_score(test_labels, inhibitors_tox_pred_RF)
precision_rf = precision_score(test_labels, inhibitors_tox_pred_RF, average=None)
recall_rf = recall_score(test_labels, inhibitors_tox_pred_RF, average=None)

print("Random forest: ")
print("Accuracy: ", accuracy_rf)
print("Precision: ", precision_rf)
print("Recall RF: ", recall_rf)
print()

#KNN model
accuracy_knn = accuracy_score(test_labels, inhibitors_tox_pred_KNN)
precision_knn = precision_score(test_labels, inhibitors_tox_pred_KNN, average=None)
recall_knn = recall_score(test_labels, inhibitors_tox_pred_KNN, average=None)

print("KNN: ")
print("Accuracy: ", accuracy_knn)
print("Precision: ", precision_knn)
print("Recall RF: ", recall_knn)
print()

#Extra trees model
accuracy_et = accuracy_score(test_labels, inhibitors_tox_pred_ET)
precision_et = precision_score(test_labels, inhibitors_tox_pred_ET, average=None)
recall_et = recall_score(test_labels, inhibitors_tox_pred_ET, average=None)

print("KNN: ")
print("Accuracy: ", accuracy_et)
print("Precision: ", precision_et)
print("Recall RF: ", recall_et)
print()

Random forest: 
Accuracy:  0.5
Precision:  [0.  0.5 0. ]
Recall RF:  [0. 1. 0.]

KNN: 
Accuracy:  0.0
Precision:  [0. 0. 0. 0.]
Recall RF:  [0. 0. 0. 0.]

KNN: 
Accuracy:  0.5
Precision:  [0.  0.5 0. ]
Recall RF:  [0. 1. 0.]



C:\Users\misch\anaconda3\envs\mipt-stats\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\misch\anaconda3\envs\mipt-stats\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\misch\anaconda3\envs\mipt-stats\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\misch\anaconda3\envs\mipt-stats\lib\site-pack

#### Реализация MyRandomForest classifier

In [2]:
import enum 
import typing as tp 
import numpy as np 
from scipy.stats import mode 

class NodeType(enum.Enum):
    REGULAR = 1
    TERMINAL = 2


def gini(y: np.ndarray) -> float:
    """
    Computes Gini index for given set of labels
    :param y: labels
    :return: Gini impurity
    """
    total_size = len(y)
    unique_labels, y_counts = np.unique(y, return_counts=True)
    gini_index = 1.0;
    for count in y_counts:
        probability = count / total_size
        gini_index -= pow(probability, 2)
    return gini_index


def weighted_impurity(y_left: np.ndarray, y_right: np.ndarray) -> \
        tp.Tuple[float, float, float]:
    """
    Computes weighted impurity by averaging children impurities
    :param y_left: left  partition
    :param y_right: right partition
    :return: averaged impurity, left child impurity, right child impurity
    """
    total_size = len(y_left) + len(y_right)
    
    unique_labels_left, y_left_counts = np.unique(y_left, return_counts=True)
    left_impurity = 1.0 - np.sum((y_left_counts / len(y_left)) ** 2)
    
    unique_labels_right, y_right_counts = np.unique(y_right, return_counts=True)
    right_impurity = 1.0 - np.sum((y_right_counts / len(y_right)) ** 2)
    
    weighted_impurity = (left_impurity * len(y_left) + right_impurity * len(y_right))/total_size
    
    return weighted_impurity, left_impurity, right_impurity


def create_split(feature_values: np.ndarray, threshold: float) -> tp.Tuple[np.ndarray, np.ndarray]:
    """
    splits given 1-d array according to relation to threshold into two subarrays
    :param feature_values: feature values extracted from data
    :param threshold: value to compare with
    :return: two sets of indices
    """
    left_idx = np.where(feature_values <= threshold)[0]
    right_idx = np.where(feature_values > threshold)[0]
    return left_idx, right_idx


class MyDecisionTreeNode:
    """
    Auxiliary class serving as representation of a decision tree node
    """

    def __init__(
            self,
            meta: 'MyDecisionTreeClassifier',
            depth,
            node_type: NodeType = NodeType.REGULAR,
            predicted_class: tp.Optional[tp.Union[int, str]] = None,
            left_subtree: tp.Optional['MyDecisionTreeNode'] = None,
            right_subtree: tp.Optional['MyDecisionTreeNode'] = None,
            feature_id: int = None,
            threshold: float = None,
            impurity: float = np.inf
    ):
        """
        :param meta: object, holding meta information about tree
        :param depth: depth of this node in a tree (is deduced on creation by depth of ancestor)
        :param node_type: 'regular' or 'terminal' depending on whether this node is a leaf node
        :param predicted_class: class label assigned to a terminal node
        :param feature_id: index if feature to split by
        :param
        """
        self._node_type = node_type
        self._meta = meta
        self._depth = depth
        self._predicted_class = predicted_class
        self._class_proba = None
        self._left_subtree = left_subtree
        self._right_subtree = right_subtree
        self._feature_id = feature_id
        self._threshold = threshold
        self._impurity = impurity

    def _best_split(self, X: np.ndarray, y: np.ndarray):
        """
        finds best split
        :param X: Data, passed to node
        :param y: labels
        :return: best feature, best threshold, left child impurity, right child impurity
        """
        lowest_impurity = np.inf
        best_feature_id = None
        best_threshold = None
        lowest_left_child_impurity, lowest_right_child_impurity = None, None
        features = self._meta.rng.permutation(X.shape[1])
        for feature in features:
            current_feature_values = X[:, feature]
            thresholds = np.unique(current_feature_values)
            for threshold in thresholds:
                left_idx, right_idx = create_split(current_feature_values, threshold)
                current_weighted_impurity, current_left_impurity, current_right_impurity = weighted_impurity(y[left_idx], y[right_idx])
                if current_weighted_impurity < lowest_impurity:
                    lowest_impurity = current_weighted_impurity
                    best_feature_id = feature
                    best_threshold = threshold
                    lowest_left_child_impurity = current_left_impurity
                    lowest_right_child_impurity = current_right_impurity

        return best_feature_id, best_threshold, lowest_left_child_impurity, lowest_right_child_impurity

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        recursively fits a node, providing it with predicted class or split condition
        :param X: Data
        :param y: labels
        :return: fitted node
        """
        if len(np.unique(y)) == 1:
            self._node_type = NodeType.TERMINAL
            self._predicted_class = y[0]
            self._class_proba = np.array([1.0])
            return self

        if self._depth >= self._meta.max_depth or len(X) < self._meta.min_samples_split:
            self._node_type = NodeType.TERMINAL
            self._predicted_class = mode(y).mode[0]
            class_counts = np.bincount(y)
            self._class_proba = class_counts / len(y)
            return self

        self._feature_id, self._threshold, left_imp, right_imp = self._best_split(X, y)
        left_idx, right_idx = create_split(X[:, self._feature_id], self._threshold)
        self._left_subtree = MyDecisionTreeNode(
            meta=self._meta,
            depth=self._depth + 1,
            impurity=left_imp
        ).fit(
            X[left_idx],
            y[left_idx]
        )
        self._right_subtree = MyDecisionTreeNode(
            meta=self._meta,
            depth=self._depth + 1,
            impurity=right_imp
        ).fit(
            X[right_idx],
            y[right_idx]
        )
        
        return self

    def predict(self, x: np.ndarray):
        """
        Predicts class for a single object
        :param x: object of shape (n_features, )
        :return: class assigned to object
        """
        if self._node_type is NodeType.TERMINAL:
            return self._predicted_class
        if x[self._feature_id] <= self._threshold:
            return self._left_subtree.predict(x)
        else:
            return self._right_subtree.predict(x)

    def predict_proba(self, x: np.ndarray):
        """
        Predicts probability for a single object
        :param x: object of shape (n_features, )
        :return: vector of probabilities assigned to object
        """
        if self._node_type is NodeType.TERMINAL:
            return self._class_proba
        if x[self._feature_id] <= self._threshold:
            return self._left_subtree.predict_proba(x)
        else:
            return self._right_subtree.predict_proba(x)


class MyDecisionTreeClassifier:
    """
    Class analogous to sklearn implementation of decision tree classifier with Gini impurity criterion,
    named in a manner avoiding collisions
    """

    def __init__(
            self,
            max_depth: tp.Optional[int] = None,
            min_samples_split: tp.Optional[int] = 2,
            seed: int = 0
    ):
        """
        :param max_depth: maximal depth of tree, prevents overfitting
        :param min_samples_split: minimal amount of samples for node to be a splitter node
        :param seed: seed for RNG, enables reproducibility
        """
        self.root = MyDecisionTreeNode(self, 1)
        self._is_trained = False
        self.max_depth = max_depth or np.inf
        self.min_samples_split = min_samples_split or 2
        self.rng = np.random.default_rng(seed)
        self._n_classes = 0

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        starts recursive process of node criterion fitting from the root
        :param X: Data
        :param y: labels
        :return: fitted self
        """
        self._n_classes = np.unique(y).shape[0]
        self.root.fit(X, y)
        self._is_trained = True
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predicts class for a sequence of objects
        :param x: Data
        :return: classes assigned to each object
        """
        if not self._is_trained:
            raise RuntimeError('predict call on untrained model')
        else:
            predictions = np.array([self.root.predict(x) for x in X])
            return predictions

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Predicts class for a sequence of objects
        :param x: Data
        :return: probabilities of all classes for each object
        """
        if not self._is_trained:
            raise RuntimeError('predict call on untrained model')
        else:
            probas = np.array([self.root.predict_proba(x) for x in X])
            return probas
        
class MyRandomForestClassifier:
    """
    Data-diverse ensemble of tree calssifiers
    """
    big_number = 1 << 32

    def __init__(
            self,
            n_estimators: int,
            max_depth: tp.Optional[int] = None,
            min_samples_split: tp.Optional[int] = 2,
            seed: int = 0
    ):
        """
        :param n_estimators: number of trees in forest
        :param max_depth: maximal depth of tree, prevents overfitting
        :param min_samples_split: minimal amount of samples for node to be a splitter node
        :param seed: seed for RNG, enables reproducibility
        """
        self._n_classes = 0
        self._is_trained = False
        self.rng = np.random.default_rng(seed)
        self.estimators = [
            MyDecisionTreeClassifier(max_depth, min_samples_split, seed=seed) for
            seed in self.rng.choice(max(MyRandomForestClassifier.big_number, n_estimators), size=(n_estimators,),
                                    replace=False)]

    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray):
        """
        returns bootstrapped sample from X of equal size
        :param X: objects collection to sample from
        :param y: corresponding labels
        :return:
        """
        indices = self.rng.choice(X.shape[0], size=X.shape[0], replace=True)
        return X[indices], y[indices]

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        fits each estimator of the ensemble on the bootstrapped data sample
        :param X: Data
        :param y: labels
        :return: fitted self
        """
        self._n_classes = np.unique(y).shape[0]
        for estimator in self.estimators:
            X_boot, y_boot = self._bootstrap_sample(X, y)
            estimator.fit(X_boot, y_boot)
        self._is_trained = True
        return self

    def predict_proba(self, X: np.ndarray):
        """
        predict probability of each class by averaging over all base estimators
        :param X: Data
        :return: array of probabilities
        """
        probas = np.zeros((X.shape[0], self._n_classes))
        for estimator in self.estimators:
            probas += estimator.predict_proba(X)
        probas /= len(self.estimators)
        return probas

    def predict(self, X):
        """
        predict class for each object
        :param X: Data
        :return: array of class labels
        """
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)